In [1]:
import pandas as pd
import re

In [90]:
launches = pd.read_csv('Launches Full.csv')
orgs = pd.read_csv('Orgs.csv')
codes = pd.read_csv('CountryCodes.csv')
satcat = pd.read_csv('satcat.csv')

In [91]:
launches['Apogee'] = pd.to_numeric(launches['Apogee'], errors = 'coerce')
launches = launches[launches['Apogee'] >=100 ] # 100 is the Karman line
launches = launches[launches['Category'] != 'Not real'] # 2 Not Real values to be ommited.
launches.LaunchCode = launches.LaunchCode.str.replace('(OS)[0-9]*', 'OS', regex = True).replace('(OF)[0-9]*', 'OF', regex = True) #Grouping Failed and successful Launches.
launches.LaunchCode.astype('category', inplace = True) 
launches.reset_index(drop = True, inplace = True)
launches = launches.merge(orgs[['StateCode', 'Class']], left_on = 'Agency', right_on = orgs.Code, how = 'left') # Adding Country and Entity identifiers from the orgs dataset
launches = launches.merge(codes['Name'], left_on = 'StateCode', right_on = codes.Code, how = 'left') # Adding country names
launches.rename(columns = {'Name' : 'Country'}, inplace = True)
print(launches.shape)

(24099, 28)


In [4]:
launches.columns

Index(['LaunchTag', 'JulianDate', 'Date', 'Year', 'LVType', 'Variant',
       'FlightId', 'Flight', 'Mission', 'FlightCode', 'Platform', 'LaunchSite',
       'LaunchPad', 'Apogee', 'Apoflag', 'Range', 'RangeFlag', 'Dest',
       'Agency', 'LaunchCode', 'Group', 'Category', 'Citation1', 'Citation2',
       'Notes', 'StateCode', 'Class', 'Country'],
      dtype='object')

In [5]:
launches[launches.Agency == 'NASA'].describe()

,JulianDate,Year,Apogee
count,2.908000e+03,2908.000000,2908.000000
mean,2.442451e+06,1974.643398,4734.163686
std,4.213130e+03,11.561195,20267.472375
min,2.436568e+06,1959.000000,100.000000
25%,2.439266e+06,1966.000000,148.000000
50%,2.441026e+06,1971.000000,200.000000
75%,2.444991e+06,1982.000000,308.000000
max,2.456132e+06,2012.000000,150000.000000


In [6]:
launches[launches.LaunchCode.isin(['OF', 'OS'])].shape

(5657, 28)

In [93]:
launches.Agency.unique().shape

(271,)

In [92]:
launches = launches.sort_values(by = 'JulianDate')

In [94]:
print(launches.describe().JulianDate['25%'], launches.describe().JulianDate['75%'], launches.describe().JulianDate['50%'])

2439348.17 2446915.9299999997 2442096.92


In [95]:
launchesBottom25 = launches[launches.JulianDate<=(launches.describe().JulianDate['25%'])]
launchesTop25 = launches[launches.JulianDate>(launches.describe().JulianDate['75%'])]
launchesFailed = launches[launches.LaunchCode.isin(['OF', 'SF', 'MF', 'HF', 'TF', 'RF'])]
launchesFailedBottom25 = launchesBottom25[launchesBottom25.LaunchCode.isin(['OF', 'SF', 'MF', 'HF', 'TF', 'RF'])]
launchesFailedTop25 = launchesTop25[launchesTop25.LaunchCode.isin(['OF', 'SF', 'MF', 'HF', 'TF', 'RF'])]

In [96]:
print(launchesFailed.shape, launchesFailedBottom25.shape, launchesFailedTop25.shape)

(821, 28) (331, 28) (205, 28)


In [97]:
launches.head(20)

,LaunchTag,JulianDate,Date,Year,LVType,Variant,FlightId,Flight,Mission,FlightCode,...,Agency,LaunchCode,Group,Category,Citation1,Citation2,Notes,StateCode,Class,Country
0,1944-S28,2431241.50,1944 Jun,1944,V-2,-,-,Vertical test,-,-,...,WEHR,MS,-,Test,Neufeld/220,-,-,DR,D,Germany
1,1944-S313,2431241.50,1944 Jun,1944,V-2,-,-,Vertical test,-,-,...,WEHR,MS,-,Test,DeMaes,-,-,DR,D,Germany
2,1944-S314,2431241.50,1944 Jun,1944,V-2,-,-,Vertical test,-,-,...,WEHR,MS,-,Test,DeMaes,-,-,DR,D,Germany
3,1944-S315,2431241.50,1944 Jun,1944,V-2,-,-,Vertical test,-,-,...,WEHR,MS,-,Test,DeMaes,-,-,DR,D,Germany
4,1944-S316,2431241.50,1944 Jun,1944,V-2,-,-,Vertical test,-,-,...,WEHR,MS,-,Test,DeMaes,-,-,DR,D,Germany
5,1944-S317,2431261.50,1944 Jun 20?,1944,V-2,-,-,Vertical test,-,-,...,WEHR,MS,-,Test,DeMaes,-,-,DR,D,Germany
6,1944-S95,2431347.50,1944 Sep 14,1944,V-2,-,-,Vertical test,-,-,...,WEHR,MS,-,Test,VonBraun58-8,-,-,DR,D,Germany
7,1944-S279,2431432.21,1944 Dec 7 1700,1944,V-2,-,19874,Ma333,-,-,...,WEHR,MS,-,Test,FE175,-,-,DR,D,Germany
8,1944-S284,2431434.22,1944 Dec 9 1710,1944,V-2,-,19020,-,-,-,...,WEHR,MS,-,Test,FE175,-,-,DR,D,Germany
9,1946-S01,2431951.39,1946 May 10 2115,1946,V-2,-,3,-,-,-,...,WSPG,SS,APL/,CR,Hermes-Rpt,-,-,US,D,United States


In [13]:
launches.Class.unique()

array(['D', 'B', 'A', 'C'], dtype=object)

In [14]:
print('B :',launches[launches.Class == 'B'].Class.shape[0], 'D :',launches[launches.Class == 'D'].Class.shape[0], 'A :',launches[launches.Class == 'A'].Class.shape[0],'C :',launches[launches.Class == 'C'].Class.shape[0],)

B : 1507 D : 15331 A : 73 C : 7188


In [15]:
launches[launches.Class == 'B'].Year.describe()

count    1507.000000
mean     1999.495687
std        16.254997
min      1946.000000
25%      1993.000000
50%      2003.000000
75%      2012.000000
max      2019.000000
Name: Year, dtype: float64

In [16]:
launches.Year.describe()

count    24099.000000
mean      1977.951824
std         15.925845
min       1944.000000
25%       1966.000000
50%       1974.000000
75%       1987.000000
max       2019.000000
Name: Year, dtype: float64

In [17]:
launches.Country.unique()

array(['Germany', 'United States', 'Russia', 'France', 'Australia',
       'United Kingdom', 'Canada', 'Japan', 'China', 'Italy', 'Turkey',
       'Pakistan', 'Sweden', 'Norway', 'European Space Agency',
       'Argentina', 'India', 'Brazil', 'Lebanon', 'Switzerland', 'Spain',
       'Denmark', 'Egypt', 'Iraq', 'Iran', 'Libya', 'Israel',
       'South Africa', 'North Korea', 'Syrian Arab Republic', 'Yemen',
       'South Korea', 'Taiwan', 'Indonesia', 'New Zealand'], dtype=object)

In [18]:
launches.Country.value_counts()[0:12]

United States     10459
Russia             8878
France              733
Germany             489
China               480
Japan               460
United Kingdom      442
India               328
Canada              297
Australia           289
Iraq                235
Spain               182
Name: Country, dtype: int64

In [19]:
launches[launches.Year >= 1999].shape

(3229, 28)

In [33]:
launches[launches.Agency == 'NASA'].Class.value_counts()

C    2908
Name: Class, dtype: int64

In [20]:
launches[launches.Year <1965].shape

(4617, 28)

In [21]:
launches[launches.Year >=1999].Country.value_counts().head(5)

United States    1309
Russia            726
China             319
France            193
India             161
Name: Country, dtype: int64

In [22]:
launches[launches.Year <1965].Country.value_counts().head(5)

United States    2520
Russia           1677
France            130
Australia         119
Canada             43
Name: Country, dtype: int64

In [87]:
launches[launches.Year == 2018].shape

(202, 28)

In [112]:
launches[launches.Year == 2018].Class.value_counts()

B    76
C    70
D    55
A     1
Name: Class, dtype: int64

In [98]:
first30 = launches[launches.Year < 1975]

In [99]:
first20 = launches[launches.Year < 1964]

In [103]:
mid20 = launches[launches.Year.between(1964 ,  1984)]

In [101]:
last30 = launches[launches.Year >1988]

In [44]:
first30.shape

(12601, 28)

In [65]:
first20.shape

(5497, 28)

In [45]:
last30.shape

(5443, 28)

In [69]:
mid20.shape

(2051, 28)

In [59]:
first20.Class.value_counts()

D    2700
C     360
A      57
B      22
Name: Class, dtype: int64

In [46]:
first30.Class.value_counts()

D    8891
C    3449
B     197
A      64
Name: Class, dtype: int64

In [47]:
last30.Class.value_counts()

D    2292
C    1895
B    1249
A       7
Name: Class, dtype: int64

In [28]:
war = launches[launches.Year<1991]
war.Class.value_counts()

D    13401
C     5502
B      307
A       66
Name: Class, dtype: int64

In [29]:
postwar = launches[launches.Year>1991]
postwar.Class.value_counts()

D    1787
C    1535
B    1178
A       7
Name: Class, dtype: int64

In [104]:

mid20.Class.value_counts()

D    8848
C    4434
B     212
A       8
Name: Class, dtype: int64

In [105]:
mid20.shape

(13502, 28)

In [108]:
satcat.shape

(44541, 13)

(19353, 13)